### Tackling the Madelon dataset!

   Let's look at two different Madelon datasets.  Our challenge is to find the relevant features.  Can I slim this down to 20 or less important features?  One problem is that the data is non-linear.  Additionally, its a bit large (one set has 500 features, another 1000 features and 200000 rows!).  Another problem is that there are lots of features that are pure noise.  

### Notebook 00: EDA

In Notebook 00, I created a Pandas DataFrame from the url from the UCI data.  Luckily the EDA required was minimal, so we could get to modeling sooner!  I then pickled the data to have easily accessible in the next notebooks.  I also did a preliminary SQL query to the large madelon dataset.  Of course I also pickled that data to use later.



### Notebook 01: Benchmark

   For Notebook 01 I began with the simple benchmark using Logistic Regression with C=100.  Here we got a 54% test set score.  Since the target value was either 1 or -1; I did a basic value count to find that the classes were very balanced--exacly 50% each in this first dataset.  So the benchmark score is barely better than guessing! 
   
   For curiosity’s sake, I also setup a few other models:  Decision Tree Classifier, Support Vector Classifier, and KNN.  I choose these because this is a Classification problem.  The first two were simply overfit, while KNN gave me a 78% train score and 74% test score with n_neighbors=17.  Not earthshaking research by any means, but a small step to get a better understanding of the data and the models.     


### Notebook 02: Pipelines!

   Next in Notebook 02 I started with building several naive pipelines to see which modeling methods had some accuracy on the smaller dataset.  I tried a few different combinations with a few different parameters.  My accuracy scores were never great; and often they were overfit.  *Ugh!*     

   To find the important features on the smaller UCI datasets, I first used Recursive Feature Elimination on each of the 10% samples.  I scaled the data and then choose n_neighbors=9.  
   
   The features are : [  2 188 209 264 328 375 377 398 452]   
   The features are : [ 10  65 181 232 240 304 309 466 475]   
   The features are : [ 40 226 229 241 248 304 388 439 444]
   
   The RFE score is : 0.72  
   The RFE score is : 0.79   
   The RFE score is : 0.77
   
   
   Not quite the accuracy I was hoping for, as each subset has little to do with each other.  Only one of those shows in multiple times [Feature 304].  

   **Second Pipeline:**

   Next I went to SelectKBest, with both f_classif and chi2 scoring functions.  This time I used the full UCI dataset for each score function.  Now I was able to see some common features:
   
   [475 241 336  64  48 105 128 378 338]   
   [475 336 105  64 493 241 453 338 442]

   **Third Pipeline:**
   
   For my next pipeline, I made a transformer pipeline to slim down to 9 features.  Here I again used SelectKBest but with the StandardScaler.  
   
   Index([48, 64, 105, 128, 241, 336, 338, 378, 475], dtype='object')   
   
   These 9 features are the same as before, with a similar method.  Scaling didn’t change much.  But I now have a reduced dataset that I can utilize later.

   **GridSearch with KNC:**
   
   Next I wanted to do a GridSearch with K Neighbors Classifier to explore feature reduction.
   
   N_neighbors=13 scored the best, but with just a 58.2 Test score:
   
   KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=13, p=2,
           weights='uniform')   
           
   Really interesting results. 13 scores the best, but not a huge difference in the scores (ie: just 1% difference in score for 7 neighbors). Interesting that 7 neighbors scores above 15, 17, and 9 neighbors. Also 3 scores above 5 neighbors!   

### Notebook 03: Big(ish) dataset.

   In Notebook 03, initially I made a smaller query of 2300 rows of data from the large data set.  This was a safe number that I could extract without freezing up my AWS.  I did a transformation pipeline to narrow down the features on the big data set.  Using SelectKBest in my pipeline, I narrowed it down to 13 features.    
   
   Index(['feat_269', 'feat_315', 'feat_341', 'feat_345', 'feat_429', 'feat_504',
       'feat_623', 'feat_681', 'feat_701', 'feat_769', 'feat_808', 'feat_829',
       'feat_920'],
      dtype='object')  
   
   Next I made a new query of 10,000 rows from the large data set, but only selecting these important 13 features.  I made a pickle of the new_huge data for use in Notebook 4 to narrow down the features even further.   

### Notebook 04:

   Into Notebook 04, first I did a simple value_counts() check of the target column values on this new_huge pickle with 10,000 rows.  Again it was very balanced with a count of 5009 and 4991 for 0 and 1, respectively.
   
   Next I made three samples of 10% of the new_huge pickle and then making test train splits for each of those samples.  Now I have some data ready to put into models.  I started with a Recursive Feature Elimination after I scaled my data.  Using the three 10% samples, I ran three models to narrow down to the selected n_features of 5.  Naturally, each gave me slightly different (yet similar) results.   
   
   The features are : [ 2  4  7 11 12]   
   The features are : [ 0  6  7 11 12]   
   The features are : [ 3  7 10 11 12]  
   
   Those indexes correspond to the following features: feat_829 (12), feat_808 (11), feat_623 (7), feat_681 (8), and feat_769 (10).  
   
   The RFE score is : 0.61   
   The RFE score is : 0.60   
   The RFE score is : 0.59  
   
   Not as accurate as I'd like, so let's try another way...


   **K Neighbors Classifier again!**
   
   I again used GridSearchCV on a KNeighborsClassifier model on the data to check for a range of n_neighbors between 3 and 13.  Here 11 neighbors scored the best, with 68.0% test score.  Further down in eighth place was 5 neighbors with a 66.7% mean test score.  
  
   My next steps would be to use GridSearchCV more deeply to get better accuracy scores.  Possibly look into more powerful models like AdaBoost and XGBoost.